## Tensor Board

Run Server for tensor board

``PS D:\PyTorch> tensorboard --logdir=runs``


we will get :TensorBoard 2.11.2 at http://localhost:6006/ (Press CTRL+C to quit)

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import sys 


# TensorBoard(1)
from torch.utils.tensorboard import SummaryWriter
# writer=SummaryWriter('runs/mnist')
writer=SummaryWriter('runs/test')

/home/basma/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyper parameters
input_size=784 #28*28
hidden_size=100
num_classes=10
num_epochs=2
batch_size=100
learning_rate=0.0001


In [3]:
# MINST
# root is where the data will be stored
# transform is how the data will be transformed --> that converts a PIL Image or numpy array into a PyTorch tensor.
# download=True will download the data if you don't have it
train_dataset=torchvision.datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=torchvision.datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor())


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [9]:
examples=iter(train_loader)
# Accessing 1 batch
samples,labels=examples.__next__()
print(samples.shape,labels.shape)

# TensorBoard(2) Plot images
# Old:
# Show Sample (1st batch)
# for i in range(6):
#     plt.subplot(2,3,i+1)
#     plt.imshow(samples[i][0],cmap='gray')
# plt.show()
# New:
img_grid=torchvision.utils.make_grid(samples)
writer.add_image('mnist_images',img_grid)


torch.Size([100, 1, 28, 28]) torch.Size([100])


In [11]:

samples,labels=examples.__next__()
img_grid=torchvision.utils.make_grid(samples)
writer.add_image('mnist_images1',img_grid)
writer.close() #to flush

In [19]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()

        # output=x*wT+bias [Linear function]
        self.l1=nn.Linear(input_size,hidden_size)

        # Activation Function
        self.relu=nn.ReLU()

        self.l2 = nn.Linear(hidden_size,num_classes)

        # Activation Function (Sigmoid) bec classification Problem but we will use cross entropy 
        # that apply that function inside it so we won't define that layer
    
    def forward(self,x):
        out=self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        return out
    
model= NeuralNet(input_size,hidden_size,num_classes).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss() # Apply Sigmoid inside it :D
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


# TensorBoard(3) Model Graph (Network)
writer.add_graph(model,samples.reshape(-1,28*28).to(device))
writer.close() #to flush


# Training
running_loss=0.0 #  To draw in tensor Board
running_correct=0 # To draw in tensor Board

n_total_steps=len(train_loader)
for epoch in range(num_epochs):
    for i , (images,labels) in enumerate(train_loader):
        # Images are 100*1*28*28
        # Reshape to 100*784
        images=images.reshape(-1,28*28).to(device) # [100,784]
        labels=labels.to(device) # [100]
  
        # Forward
        outputs=model(images)
        loss=criterion(outputs,labels)

        # Backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # For Tensor Board
        running_loss+=loss.item()
        _,predicted=torch.max(outputs.data,1)
        running_correct += (predicted == labels).sum().item()

        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps} , loss = {loss.item()} ')
            
            # TensorBoard(4) To add Scalar for the loss to draw training Graph
            writer.add_scalar('training loss',running_loss/100,epoch*n_total_steps + i) #/100 bec we are running every 100 step
            writer.add_scalar('accuracy',running_correct/100,epoch*n_total_steps + i) #/100 bec we are running every 100 step
            running_loss=0.0
            running_correct=0


epoch 1/2, step 100/600 , loss = 1.8197262287139893 
epoch 1/2, step 200/600 , loss = 1.2843384742736816 
epoch 1/2, step 300/600 , loss = 0.9366411566734314 
epoch 1/2, step 400/600 , loss = 0.7127205729484558 
epoch 1/2, step 500/600 , loss = 0.6208563446998596 
epoch 1/2, step 600/600 , loss = 0.6254121661186218 
epoch 2/2, step 100/600 , loss = 0.525275468826294 
epoch 2/2, step 200/600 , loss = 0.4228530824184418 
epoch 2/2, step 300/600 , loss = 0.4626455307006836 
epoch 2/2, step 400/600 , loss = 0.4188263714313507 
epoch 2/2, step 500/600 , loss = 0.4544052481651306 
epoch 2/2, step 600/600 , loss = 0.4532298147678375 


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()

        # output=x*wT+bias [Linear function]
        self.l1=nn.Linear(input_size,hidden_size)

        # Activation Function
        self.relu=nn.ReLU()

        self.l2 = nn.Linear(hidden_size,num_classes)

        # Activation Function (Sigmoid) bec classification Problem but we will use cross entropy 
        # that apply that function inside it so we won't define that layer
    
    def forward(self,x):
        out=self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        return out
    
model= NeuralNet(input_size,hidden_size,num_classes).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss() # Apply Sigmoid inside it :D
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


# Training
n_total_steps=len(train_loader)
for epoch in range(num_epochs):
    for i , (images,labels) in enumerate(train_loader):
        # Images are 100*1*28*28
        # Reshape to 100*784
        images=images.reshape(-1,28*28).to(device) # [100,784]
        labels=labels.to(device) # [100]
  
        # Forward
        outputs=model(images)
        loss=criterion(outputs,labels)

        # Backwards
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps} , loss = {loss.item()} ')


epoch 1/2, step 100/600 , loss = 0.48600172996520996 
epoch 1/2, step 200/600 , loss = 0.3482045829296112 
epoch 1/2, step 300/600 , loss = 0.3221309185028076 
epoch 1/2, step 400/600 , loss = 0.24100519716739655 
epoch 1/2, step 500/600 , loss = 0.18110185861587524 
epoch 1/2, step 600/600 , loss = 0.24509309232234955 
epoch 2/2, step 100/600 , loss = 0.22404764592647552 
epoch 2/2, step 200/600 , loss = 0.22133058309555054 
epoch 2/2, step 300/600 , loss = 0.27536407113075256 
epoch 2/2, step 400/600 , loss = 0.3860691487789154 
epoch 2/2, step 500/600 , loss = 0.09852485358715057 
epoch 2/2, step 600/600 , loss = 0.11960834264755249 


In [45]:

# Testing
with torch.no_grad():
    n_correct=0
    n_samples=0
    for images,labels in test_loader:
        images=images.reshape(-1,28*28).to(device)
        labels=labels.to(device)

        outputs=model(images)

        # print(outputs.shape) #torch.Size([100, 10])
        # value,prediction
        _,predictions=torch.max(outputs,1) #on dim 1 which is the 10 classes
        
        n_samples+=labels.shape[0]
        n_correct+=(predictions==labels).sum().item() #.item() bec it is tensor he want value
    acc =100.0* n_correct/n_samples
    print(f'accuracy = {acc}')

accuracy = 95.53
